In [39]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
import re
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import re

In [11]:
data = pd.read_csv('processed_df.csv')

# **ПОПРОБУЕМ ОБУЧИТЬ МОДЕЛЬ НА ДАННЫХ**

**ВЫКИНЕМ НАНЫ**

In [16]:
data_no_nans = data.copy()
data_no_nans = data_no_nans.dropna()
data_no_nans.isna().sum()

Unnamed: 0              0
Maruti                  0
Skoda                   0
Honda                   0
Hyundai                 0
Toyota                  0
Ford                    0
Renault                 0
Mahindra                0
Tata                    0
Chevrolet               0
Fiat                    0
Datsun                  0
Jeep                    0
Mercedes-Benz           0
Mitsubishi              0
Audi                    0
Volkswagen              0
BMW                     0
Nissan                  0
Lexus                   0
Jaguar                  0
Land                    0
MG                      0
Volvo                   0
Daewoo                  0
Kia                     0
Force                   0
Ambassador              0
Ashok                   0
Isuzu                   0
Opel                    0
year                    0
selling_price           0
km_driven               0
Diesel                  0
Individual              0
Dealer                  0
Manual      

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЕ И ТЕСТОВЫЕ МАССИВЫ**

In [27]:
train_data_no_nans = data_no_nans.sample(frac=0.8)
test_data_no_nans = data_no_nans.iloc[~data_no_nans.index.isin(train_data_no_nans.index), :]

**НОРМИРУЕМ ДАННЫЕ**

In [30]:
features = train_data_no_nans.iloc[:, train_data_no_nans.columns != 'selling_price']
responses = train_data_no_nans['selling_price'].to_frame(name='selling_price')

In [31]:
scaller = StandardScaler()
x = scaller.fit_transform(features)
y = scaller.fit_transform(responses)

**ОБУЧИМ МОДЕЛЬ С ЛИНЕАРИЗАЦИЕЙ КОЭФФИЦИЕНТОВ L1 LASSO**

In [46]:
l1_regr = Ridge(alpha=0.000000001)
l1_regr.fit(x, y)
l1_regr.score(x, y)

0.8512069325918739